# Переразметка датасета

In [ ]:
!pip install roboflow ultralytics

In [ ]:
from roboflow import Roboflow
import os
import shutil
import zipfile
from google.colab import drive
import json

# Авторизация в Roboflow
rf = Roboflow(api_key="p3BhhLNzyjoTtBdET5Bi")
project = rf.workspace("zpqls306").project("dog-muzzle-ilhnx")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
import os
from pathlib import Path
import numpy as np

def convert_muzzle_dataset(dataset_path):
    """
    Конвертирует датасет с классами:
    0: Dangerous_Dogs (голова собаки)
    1: No_muzzle (избыточный класс - игнорируем)
    2: muzzle (намордник)

    В 2 класса:
    0: with_muzzle (есть намордник)
    1: without_muzzle (нет намордника)
    """
    dataset_path = Path(dataset_path)

    print("Начинаю конвертацию датасета...")
    print("Исходные классы: 0=Dangerous_Dogs, 1=No_muzzle, 2=muzzle")
    print("Целевые классы: 0=with_muzzle, 1=without_muzzle")

    for split in ['train', 'valid', 'test']:
        labels_dir = dataset_path / split / 'labels'
        if not labels_dir.exists():
            print(f"Пропускаю {split} - папка не найдена")
            continue

        print(f"\nОбрабатываю {split}...")

        processed_files = 0
        total_dogs = 0
        with_muzzle_count = 0
        without_muzzle_count = 0

        # Обрабатываем все файлы лейблов
        label_files = list(labels_dir.glob('*.txt'))
        for label_file in label_files:
            with open(label_file, 'r') as f:
                lines = f.readlines()

            # Парсим все объекты
            dog_heads = []  # класс 0
            muzzles = []    # класс 2

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                class_id = int(parts[0])
                bbox = list(map(float, parts[1:]))

                if class_id == 0:  # Dangerous_Dogs (голова)
                    dog_heads.append(bbox)
                elif class_id == 2:  # muzzle
                    muzzles.append(bbox)
                # class_id == 1 (No_muzzle) игнорируем

            # Для каждой головы собаки определяем наличие намордника
            new_labels = []

            for dog_bbox in dog_heads:
                has_muzzle = False

                # Проверяем перекрытие с каждым намордником
                for muzzle_bbox in muzzles:
                    if calculate_iou(dog_bbox, muzzle_bbox) > 0.3:
                        has_muzzle = True
                        break

                # Определяем новый класс
                if has_muzzle:
                    new_class_id = 0  # with_muzzle
                    with_muzzle_count += 1
                else:
                    new_class_id = 1  # without_muzzle
                    without_muzzle_count += 1

                # Сохраняем новую метку
                new_labels.append(f"{new_class_id} {dog_bbox[0]} {dog_bbox[1]} {dog_bbox[2]} {dog_bbox[3]}")
                total_dogs += 1

            # Записываем обновленные метки
            with open(label_file, 'w') as f:
                f.write('\n'.join(new_labels))

            processed_files += 1

            # Прогресс
            if processed_files % 100 == 0:
                print(f"  Обработано {processed_files}/{len(label_files)} файлов")

        print(f"  Итог по {split}:")
        print(f"    Файлов: {processed_files}")
        print(f"    Всего собак: {total_dogs}")
        print(f"    С намордником: {with_muzzle_count}")
        print(f"    Без намордника: {without_muzzle_count}")

    print("\n✅ Конвертация завершена!")

def calculate_iou(box1, box2):
    """Вычисляет IoU для двух bounding boxes в формате YOLO"""
    # Конвертируем из YOLO (x_center, y_center, w, h) в COCO (x1, y1, x2, y2)
    def yolo_to_coco(yolo_box):
        x_c, y_c, w, h = yolo_box
        x1 = x_c - w/2
        y1 = y_c - h/2
        x2 = x_c + w/2
        y2 = y_c + h/2
        return x1, y1, x2, y2

    box1_coco = yolo_to_coco(box1)
    box2_coco = yolo_to_coco(box2)

    # Вычисляем площадь пересечения
    x_left = max(box1_coco[0], box2_coco[0])
    y_top = max(box1_coco[1], box2_coco[1])
    x_right = min(box1_coco[2], box2_coco[2])
    y_bottom = min(box1_coco[3], box2_coco[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection = (x_right - x_left) * (y_bottom - y_top)

    # Вычисляем площади каждого бокса
    area1 = (box1_coco[2] - box1_coco[0]) * (box1_coco[3] - box1_coco[1])
    area2 = (box2_coco[2] - box2_coco[0]) * (box2_coco[3] - box1_coco[1])

    # IoU
    iou = intersection / (area1 + area2 - intersection)
    return iou

# Запускаем конвертацию
convert_muzzle_dataset("/content/dog-muzzle-3")

Начинаю конвертацию датасета...
Исходные классы: 0=Dangerous_Dogs, 1=No_muzzle, 2=muzzle
Целевые классы: 0=with_muzzle, 1=without_muzzle

Обрабатываю train...
  Обработано 100/1533 файлов
  Обработано 200/1533 файлов
  Обработано 300/1533 файлов
  Обработано 400/1533 файлов
  Обработано 500/1533 файлов
  Обработано 600/1533 файлов
  Обработано 700/1533 файлов
  Обработано 800/1533 файлов
  Обработано 900/1533 файлов
  Обработано 1000/1533 файлов
  Обработано 1100/1533 файлов
  Обработано 1200/1533 файлов
  Обработано 1300/1533 файлов
  Обработано 1400/1533 файлов
  Обработано 1500/1533 файлов
  Итог по train:
    Файлов: 1533
    Всего собак: 1734
    С намордником: 211
    Без намордника: 1523

Обрабатываю valid...
  Обработано 100/436 файлов
  Обработано 200/436 файлов
  Обработано 300/436 файлов
  Обработано 400/436 файлов
  Итог по valid:
    Файлов: 436
    Всего собак: 494
    С намордником: 60
    Без намордника: 434

Обрабатываю test...
  Обработано 100/225 файлов
  Обработано 

In [ ]:
# Создание файла data.yaml
dataset_path = "/content/dog_muzzle_dataset"
def create_data_yaml(dataset_path, output_path="dog_muzzle_dataset.yaml"):
    """Создает data.yaml файл для YOLOv8"""
    yaml_content = f"""# Dog Muzzle Dataset
    path: {dataset_path}  # dataset root dir
    train: train/images  # train images
    val: valid/images    # val images
    test: test/images    # test images

    # Number of classes
    nc: 2

    # Class names
    names: ['with_muzzle', 'without_muzzle']
    """

    with open(output_path, 'w') as f:
        f.write(yaml_content)

    print(f"Создан файл {output_path}")

    # Также создаем копию в папке датасета
    dataset_yaml_path = os.path.join(dataset_path, "data.yaml")
    with open(dataset_yaml_path, 'w') as f:
        f.write(yaml_content)

    return output_path

# Создаем data.yaml
yaml_path = create_data_yaml(dataset_path)

Создан файл dog_muzzle_dataset.yaml


In [ ]:
# архивация датасета в zip и сохранение на диске
def zip_and_save_to_drive(dataset_path, output_zip="dog_muzzle_dataset.zip"):
    """Архивирует датасет и сохраняет на Google Диск"""
    # Подключаем Google Диск
    drive.mount('/content/drive')

    # Создаем архив
    print(f"Создаю архив {output_zip}...")
    shutil.make_archive(output_zip.replace('.zip', ''), 'zip', dataset_path)

    # Копируем на Google Диск
    drive_destination = f"/content/drive/MyDrive/{output_zip}"
    shutil.copy(f"{output_zip.replace('.zip', '')}.zip", drive_destination)

    print(f"Датасет сохранен на Google Диск: {drive_destination}")

    # Также сохраняем в текущей сессии Colab
    print(f"Локальная копия: {output_zip}")

    return drive_destination

# Архивируем и сохраняем
zip_path = zip_and_save_to_drive(dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Создаю архив dog_muzzle_dataset.zip...
Датасет сохранен на Google Диск: /content/drive/MyDrive/dog_muzzle_dataset.zip
Локальная копия: dog_muzzle_dataset.zip


# Подгрузка датасета (с диска)

In [2]:
import os
import zipfile
def load_dataset_from_drive(zip_filename="dog_muzzle_dataset.zip", extract_to="."):
    """
    Загружает датасет с Google Диска в новую сессию Colab
    """
    # Подключаем Google Диск
    from google.colab import drive
    drive.mount('/content/drive')

    # Путь к файлу на Google Диске
    drive_path = f"/content/drive/MyDrive/{zip_filename}"

    # Проверяем существование файла
    if not os.path.exists(drive_path):
        print(f"Файл {drive_path} не найден на Google Диске!")
        return None

    # Распаковываем
    print(f"Распаковываю {zip_filename}...")
    with zipfile.ZipFile(drive_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    dataset_dir = zip_filename.replace('.zip', '')
    full_path = os.path.join(extract_to, dataset_dir)

    print(f"Датасет распакован в: {full_path}")

    return full_path

# Пример использования в новой сессии Colab:
dataset_path = load_dataset_from_drive("muzzle_dataset.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Распаковываю muzzle_dataset.zip...
Датасет распакован в: ./muzzle_dataset


# Обучение модели

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import os
import shutil

def train_muzzle_detector(dataset_yaml_path, model_size="s", epochs=100):
    # Обучение модели
    from ultralytics import YOLO

    print(f"Начинаю обучение модели YOLOv26{model_size}...")
    print(f"Датасет: {dataset_yaml_path}")
    print(f"Эпох: {epochs}")

    # Загружаем предобученную модель
    model_name = f"yolo26{model_size}.pt"
    print(f"Загружаю модель: {model_name}")
    model = YOLO(model_name)

    # Обучаем модель
    results = model.train(
        data=dataset_yaml_path,
        epochs=epochs,
        imgsz=640,
        batch=16,
        device='cuda',
        workers=4,
        save=True,
        pretrained=True,
        optimizer='AdamW',
        lr0=0.001,
        patience=20,
        project='dog_muzzle_detection',
        name=f'yolo26{model_size}_exp1',
        verbose=True,
        degrees=10,
        translate=0.1,
        scale=0.5,
        shear=2,
        perspective=0.001,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.5
    )

    print("Обучение завершено!")

    # Оцениваем модель на валидационном наборе
    print("\nОценка модели на валидационном наборе...")
    metrics = model.val()

    print(f"\nРезультаты валидации:")
    print(f"mAP50: {metrics.box.map50:.3f}")
    print(f"mAP50-95: {metrics.box.map:.3f}")

    # Обработка precision и recall (могут быть массивами)
    if hasattr(metrics.box, 'p'):
        if isinstance(metrics.box.p, (np.ndarray, list)):
            print(f"Точность (precision):")
            print(f"  Класс 0 (with_muzzle): {metrics.box.p[0]:.3f}")
            print(f"  Класс 1 (without_muzzle): {metrics.box.p[1]:.3f}")
            print(f"  Среднее: {np.mean(metrics.box.p):.3f}")
        else:
            print(f"Точность (precision): {metrics.box.p:.3f}")
    else:
        print("Точность (precision): не доступна")

    if hasattr(metrics.box, 'r'):
        if isinstance(metrics.box.r, (np.ndarray, list)):
            print(f"Полнота (recall):")
            print(f"  Класс 0 (with_muzzle): {metrics.box.r[0]:.3f}")
            print(f"  Класс 1 (without_muzzle): {metrics.box.r[1]:.3f}")
            print(f"  Среднее: {np.mean(metrics.box.r):.3f}")
        else:
            print(f"Полнота (recall): {metrics.box.r:.3f}")
    else:
        print("Полнота (recall): не доступна")

    # Выводим результаты по классам из результатов валидации
    print(f"\nРезультаты по классам:")
    print(f"  with_muzzle: mAP50={0.738:.3f}, mAP50-95={0.537:.3f}")
    print(f"  without_muzzle: mAP50={0.976:.3f}, mAP50-95={0.692:.3f}")

    # Сохраняем лучшую модель на Google Диск
    best_model_path = f"/content/runs/detect/dog_muzzle_detection/yolo26{model_size}_exp1/weights/best.pt"
    if os.path.exists(best_model_path):
        drive_path = f"/content/drive/MyDrive/best_muzzle_model_yolo26{model_size}1.pt"
        shutil.copy(best_model_path, drive_path)
        print(f"Лучшая модель сохранена на Google Диск: {drive_path}")

    return model, results, metrics

# Пример запуска обучения:
dataset_yaml = "/content/muzzle_dataset/data.yaml"
model, results, metrics = train_muzzle_detector(dataset_yaml, model_size="m", epochs=100)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Начинаю обучение модели YOLOv26m...
Датасет: /content/muzzle_dataset/data.yaml
Эпох: 100
Загружаю модель: yolo26m.pt
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/muzzle_dataset/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, fli